# Standard Data Science Workflow

A train/test split of the Iris dataset followed by generating feature matrices and encoding the target classes into integers, then some simple model fitting and finally creating a confusion matrix.

In [2]:
import random
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import sqltables

In [3]:
# Some data mangling with list/dictionary data structures. 
# Helps to put the relational code below in context ;)

iris = load_iris()
feature_rows = iris["data"].tolist()
labels = [iris["target_names"][t] for t in iris["target"].tolist()]
rows = [[i, t, *r] for i, (t, r) in enumerate(zip(labels, feature_rows))]
column_names = ["id", "target", *iris["feature_names"]]

In [4]:
# Create an in-memory database
db = sqltables.Database()

# Register the Python random() function for use with SQLite
db.create_function("random1", 0, random.random)

In [5]:
# Load the Iris data into an SQL table
input = db.load_values(rows, column_names=column_names)

In [6]:
# Display implemented through the _repr_markdown_() method on Table
input

|id|target|sepal length \(cm\)|sepal width \(cm\)|petal length \(cm\)|petal width \(cm\)|
|-|-|-|-|-|-|
|0|\'setosa\'|5\.1|3\.5|1\.4|0\.2|
|1|\'setosa\'|4\.9|3\.0|1\.4|0\.2|
|2|\'setosa\'|4\.7|3\.2|1\.3|0\.2|
|3|\'setosa\'|4\.6|3\.1|1\.5|0\.2|
|4|\'setosa\'|5\.0|3\.6|1\.4|0\.2|
|5|\'setosa\'|5\.4|3\.9|1\.7|0\.4|
|6|\'setosa\'|4\.6|3\.4|1\.4|0\.3|
|7|\'setosa\'|5\.0|3\.4|1\.5|0\.2|
|8|\'setosa\'|4\.4|2\.9|1\.4|0\.2|
|9|\'setosa\'|4\.9|3\.1|1\.5|0\.1|
|10|\'setosa\'|5\.4|3\.7|1\.5|0\.2|
|11|\'setosa\'|4\.8|3\.4|1\.6|0\.2|
|12|\'setosa\'|4\.8|3\.0|1\.4|0\.1|
|13|\'setosa\'|4\.3|3\.0|1\.1|0\.1|
|14|\'setosa\'|5\.8|4\.0|1\.2|0\.2|
|15|\'setosa\'|5\.7|4\.4|1\.5|0\.4|
|...|...|...|...|...|...|


In [7]:
# Create train/test split

random.seed(23)
subsets = input.table("""
select 
    id, 
    case random1() <= 0.25
        when true then 'test' 
        else 'train' 
    end as subset 
    from _
""")
subsets

|id|subset|
|-|-|
|0|\'train\'|
|1|\'train\'|
|2|\'train\'|
|3|\'test\'|
|4|\'train\'|
|5|\'train\'|
|6|\'train\'|
|7|\'test\'|
|8|\'test\'|
|9|\'train\'|
|10|\'test\'|
|11|\'train\'|
|12|\'test\'|
|13|\'test\'|
|14|\'train\'|
|15|\'train\'|
|...|...|


In [8]:
# Sanity check
subsets.view("select subset, count(*) from _ group by subset")

|subset|count\(\*\)|
|-|-|
|\'test\'|36|
|\'train\'|114|


In [9]:
# Define a function to create a feature matrix from an Iris dataset

def extract_feature_matrix(dataset):
    features = dataset.view("""
    select 
        "sepal length (cm)",
        "sepal width (cm)",
        "petal length (cm)",
        "petal width (cm)"
    from _
    order by id
    """)
    return np.array(list(features))

def extract_target_indices(dataset, target_encoding):
    data_encoded = dataset.view("""
        select encoding from _ join target_encoding using (target)
        order by id
    """, bindings={"target_encoding": target_encoding})
    return np.array([enc for [enc] in data_encoded])

def extract_ids(dataset):
    return [id for [id] in dataset.view("select id from _ order by id")]


In [10]:
# Create a dictionary for the class labels

target_encoding = input.table("""
    select target, row_number() over (order by target) as encoding
    from (select distinct target from _)
""")
target_encoding

|target|encoding|
|-|-|
|\'setosa\'|1|
|\'versicolor\'|2|
|\'virginica\'|3|


In [11]:
# Create dictionaries ids, X and y with ids, feature and target data 
# for each subset. The ids can be used for example to link the predictions
# of the model back to the relational data.
# We are leaving the relational domain here and enter the number
# crunching realm.

X = {}
y = {}
ids = {}

for [subset] in subsets.view("select distinct subset from _"):
    bindings = {"subsets": subsets}
    subset_data = input.table("""
        select * from _ join subsets using (id) 
        where subset = ?
        order by id
    """, [subset], bindings=bindings)
    X[subset] = extract_feature_matrix(subset_data)
    y[subset] = extract_target_indices(subset_data, target_encoding)
    ids[subset] = extract_ids(subset_data)

print({k: v.shape for k, v in X.items()})
print({k: v.shape for k, v in y.items()})
print({k: len(v) for k, v in ids.items()})

{'train': (114, 4), 'test': (36, 4)}
{'train': (114,), 'test': (36,)}
{'train': 114, 'test': 36}


In [12]:
# Fit a model and make some predictions. 
# Mangle the predictions back into a table.

classifier = LogisticRegression(
    multi_class='multinomial',  solver="newton-cg"
)
classifier.fit(X["train"], y["train"])
input_ids = [id for [id] in input.view("select id from _ order by id")]
y_hat = classifier.predict(extract_feature_matrix(input)).tolist()
rows = [[id, y1] for id, y1 in zip(input_ids, y_hat)]
predictions_encoded = db.create_table(
    values=rows, column_names=["id", "encoding"])

# Back into the relational world
predictions = predictions_encoded.table("""
    select id, target_encoding.target as prediction 
    from _ 
    join target_encoding using (encoding)
    """, bindings={"target_encoding": target_encoding})

predictions

|id|prediction|
|-|-|
|0|\'setosa\'|
|1|\'setosa\'|
|2|\'setosa\'|
|3|\'setosa\'|
|4|\'setosa\'|
|5|\'setosa\'|
|6|\'setosa\'|
|7|\'setosa\'|
|8|\'setosa\'|
|9|\'setosa\'|
|10|\'setosa\'|
|11|\'setosa\'|
|12|\'setosa\'|
|13|\'setosa\'|
|14|\'setosa\'|
|15|\'setosa\'|
|...|...|


In [13]:
# Aggregate data into a single table for convenient analysis
result = input.table("""
    select * 
    from _ 
    join predictions using (id)
    join subsets using (id)
    """, bindings={"predictions": predictions, "subsets": subsets})
result

|id|target|sepal length \(cm\)|sepal width \(cm\)|petal length \(cm\)|petal width \(cm\)|prediction|subset|
|-|-|-|-|-|-|-|-|
|0|\'setosa\'|5\.1|3\.5|1\.4|0\.2|\'setosa\'|\'train\'|
|1|\'setosa\'|4\.9|3\.0|1\.4|0\.2|\'setosa\'|\'train\'|
|2|\'setosa\'|4\.7|3\.2|1\.3|0\.2|\'setosa\'|\'train\'|
|3|\'setosa\'|4\.6|3\.1|1\.5|0\.2|\'setosa\'|\'test\'|
|4|\'setosa\'|5\.0|3\.6|1\.4|0\.2|\'setosa\'|\'train\'|
|5|\'setosa\'|5\.4|3\.9|1\.7|0\.4|\'setosa\'|\'train\'|
|6|\'setosa\'|4\.6|3\.4|1\.4|0\.3|\'setosa\'|\'train\'|
|7|\'setosa\'|5\.0|3\.4|1\.5|0\.2|\'setosa\'|\'test\'|
|8|\'setosa\'|4\.4|2\.9|1\.4|0\.2|\'setosa\'|\'test\'|
|9|\'setosa\'|4\.9|3\.1|1\.5|0\.1|\'setosa\'|\'train\'|
|10|\'setosa\'|5\.4|3\.7|1\.5|0\.2|\'setosa\'|\'test\'|
|11|\'setosa\'|4\.8|3\.4|1\.6|0\.2|\'setosa\'|\'train\'|
|12|\'setosa\'|4\.8|3\.0|1\.4|0\.1|\'setosa\'|\'test\'|
|13|\'setosa\'|4\.3|3\.0|1\.1|0\.1|\'setosa\'|\'test\'|
|14|\'setosa\'|5\.8|4\.0|1\.2|0\.2|\'setosa\'|\'train\'|
|15|\'setosa\'|5\.7|4\.4|1\.5|0\.4|\'setosa\'|\'train\'|
|...|...|...|...|...|...|...|...|


In [14]:
# Confusion matrix

result.view("""
select
    subset, target, prediction, count(*) as count 
from _ 
group by subset, target, prediction 
order by subset, target, prediction
""")

|subset|target|prediction|count|
|-|-|-|-|
|\'test\'|\'setosa\'|\'setosa\'|14|
|\'test\'|\'versicolor\'|\'versicolor\'|10|
|\'test\'|\'versicolor\'|\'virginica\'|1|
|\'test\'|\'virginica\'|\'virginica\'|11|
|\'train\'|\'setosa\'|\'setosa\'|36|
|\'train\'|\'versicolor\'|\'versicolor\'|37|
|\'train\'|\'versicolor\'|\'virginica\'|2|
|\'train\'|\'virginica\'|\'virginica\'|39|


In [15]:
# Inspect

result.view("select * from _ where target != prediction")

|id|target|sepal length \(cm\)|sepal width \(cm\)|petal length \(cm\)|petal width \(cm\)|prediction|subset|
|-|-|-|-|-|-|-|-|
|70|\'versicolor\'|5\.9|3\.2|4\.8|1\.8|\'virginica\'|\'train\'|
|77|\'versicolor\'|6\.7|3\.0|5\.0|1\.7|\'virginica\'|\'train\'|
|83|\'versicolor\'|6\.0|2\.7|5\.1|1\.6|\'virginica\'|\'test\'|


In [16]:
# Write the output to the file result.sqlite3

output_db = sqltables.Database("result.sqlite3")
it = iter(result)
output_db.create_table(values=it, column_names=it.column_names, name="result")

|id|target|sepal length \(cm\)|sepal width \(cm\)|petal length \(cm\)|petal width \(cm\)|prediction|subset|
|-|-|-|-|-|-|-|-|
|0|\'setosa\'|5\.1|3\.5|1\.4|0\.2|\'setosa\'|\'train\'|
|1|\'setosa\'|4\.9|3\.0|1\.4|0\.2|\'setosa\'|\'train\'|
|2|\'setosa\'|4\.7|3\.2|1\.3|0\.2|\'setosa\'|\'train\'|
|3|\'setosa\'|4\.6|3\.1|1\.5|0\.2|\'setosa\'|\'test\'|
|4|\'setosa\'|5\.0|3\.6|1\.4|0\.2|\'setosa\'|\'train\'|
|5|\'setosa\'|5\.4|3\.9|1\.7|0\.4|\'setosa\'|\'train\'|
|6|\'setosa\'|4\.6|3\.4|1\.4|0\.3|\'setosa\'|\'train\'|
|7|\'setosa\'|5\.0|3\.4|1\.5|0\.2|\'setosa\'|\'test\'|
|8|\'setosa\'|4\.4|2\.9|1\.4|0\.2|\'setosa\'|\'test\'|
|9|\'setosa\'|4\.9|3\.1|1\.5|0\.1|\'setosa\'|\'train\'|
|10|\'setosa\'|5\.4|3\.7|1\.5|0\.2|\'setosa\'|\'test\'|
|11|\'setosa\'|4\.8|3\.4|1\.6|0\.2|\'setosa\'|\'train\'|
|12|\'setosa\'|4\.8|3\.0|1\.4|0\.1|\'setosa\'|\'test\'|
|13|\'setosa\'|4\.3|3\.0|1\.1|0\.1|\'setosa\'|\'test\'|
|14|\'setosa\'|5\.8|4\.0|1\.2|0\.2|\'setosa\'|\'train\'|
|15|\'setosa\'|5\.7|4\.4|1\.5|0\.4|\'setosa\'|\'train\'|
|...|...|...|...|...|...|...|...|
